In [3]:
# !pip install "langchain[all]"

Question
- Answer
    - source 1 (small piece of the pdf document)
    - source 2 (small piece of the pdf document)
    - source 3 (small piece of the pdf document)

In [1]:
!wget https://arxiv.org/pdf/2212.14024.pdf

--2024-03-11 11:24:40--  https://arxiv.org/pdf/2212.14024.pdf
Resolving arxiv.org (arxiv.org)... 151.101.3.42, 151.101.195.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.3.42|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1268099 (1,2M) [application/pdf]
Saving to: ‘2212.14024.pdf’

2212.14024.pdf      100%[===================>]   1,21M  --.-KB/s    in 0,04s   

2024-03-11 11:24:40 (33,4 MB/s) - ‘2212.14024.pdf’ saved [1268099/1268099]



In [24]:
# !pip install langchain-openai
# !pip install pypdf
# !pip install chromadb

In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

In [2]:
loader = PyPDFLoader("./rag-intro-paper.pdf")

In [3]:
documents = loader.load_and_split()

In [4]:
for i, doc in enumerate(documents):
    doc.metadata["page_chunk"] = i

In [5]:
embeddings = OpenAIEmbeddings()

In [6]:
vectorstore = Chroma.from_documents(documents, embeddings, collection_name="structured-answer-from-pdf")

In [7]:
retriever = vectorstore.as_retriever()

In [8]:
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x12a3f5540>)

In [10]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "structured-answer-from-pdf",
    "Query a retriever to get information about a rag pipeline in NLP intensive tasks from a pdf.",
)

In [11]:
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List

class Answer(BaseModel):
    """Final response to the question being asked"""

    answer: str = Field(description="The final answer to respond to the user")
    sources: List[int] = Field(
        description="List of page chunks that contain answer to the question. Only include a page chunk if it contains relevant information"
    )



# Agent Loop

Run time of an agent involves:

while agent is running:
   1. get input from user
   2. return response or make a fucntion call (take an action)
   3. If function call: 
       3.1 observation
   4. function_of_observation(observation):
   5. repet from step 2   

- Handling erorrs
- parsing  output from functions

In [12]:
import json

from langchain_core.agents import AgentActionMessageLog, AgentFinish

In [13]:
def parse(output):
    # If no function was invoked, return to user
    if "function_call" not in output.additional_kwargs:
        return AgentFinish(return_values={"output": output.content}, log=output.content)

    # Parse out the function call
    function_call = output.additional_kwargs["function_call"]
    name = function_call["name"]
    inputs = json.loads(function_call["arguments"])

    # If the Response function was invoked, return to the user with the function inputs
    if name == "Answer":
        return AgentFinish(return_values=inputs, log=str(function_call))
    # Otherwise, return an agent action
    else:
        return AgentActionMessageLog(
            tool=name, tool_input=inputs, log="", message_log=[output]
        )

In [14]:
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        ("user", "{input}"),
        
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [16]:
llm = ChatOpenAI(temperature=0, model="gpt-4-turbo-preview")

In [17]:
llm_with_tools = llm.bind_functions([retriever_tool, Answer])

LCEL is a declarative expression language that allows for easy composition of chains with streaming support, async capabilities, optimized parallel execution, and configurable retries and fallbacks.

In [18]:
agent = (
    {
        "input": lambda x: x["input"],
        # Format agent scratchpad from intermediate steps
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | parse
)

In [19]:
agent_executor = AgentExecutor(tools=[retriever_tool], agent=agent, verbose=True)

In [20]:
query = "Explain simply the Retrieval Model for the DSP framework."
output = agent_executor.invoke(
    {"input": query},
    return_only_outputs=True,
)
output



> Entering new AgentExecutor chain...
As a general framework, DSP can express ideas like self-ask
and many other, more sophisticated pipelines as we discuss
in the present section. More importantly, DSP offers a num-
ber of intrinsic advantages that lead to large empirical gains:
80%–290% over self-ask. For instance, DSP programs are
deeply modular, which among other things means that DSP
programs will annotate and construct their own demonstra-
tions. Thus, they can be developed without labeling any
of the intermediate transformations (e.g., the queries gener-
ated). In addition, the LM prompts constructed by DSP get
automatically updated to align with the training data and re-
trieval corpus provided. In contrast, approaches like self-ask
rely on a hand-written prompt with hard-coded examples.
Moreover, DSP assigns the control ﬂow to an explicit pro-
gram and facilitates design patterns that invoke the LM(or
RM) to conduct small transformations. This allows us to
build steps that a

{'answer': 'The Retrieval Model (RM) within the DSP (DEMONSTRATE – SEARCH – PREDICT) framework plays a crucial role in enhancing the capabilities of language models (LMs) for knowledge-intensive tasks. The DSP framework is designed to facilitate sophisticated interactions between LMs and RMs, allowing for the construction of complex, reliable pipelines that systematically break down problems into smaller, manageable transformations. This modular approach enables DSP programs to generate retrieval queries, summarize multiple passages, and provide answers more effectively than simpler retrieve-then-read pipelines or self-ask approaches. By assigning explicit control flow to DSP programs and leveraging the RM for both retrieving relevant passages and constructing demonstrations, DSP achieves significant empirical gains in performance. Additionally, the framework supports the fusion of retrieval results from multiple queries to improve recall and robustness, showcasing the power of combini

In [21]:
from IPython.display import display, Markdown

Markdown(output["answer"])

The Retrieval Model (RM) within the DSP (DEMONSTRATE – SEARCH – PREDICT) framework plays a crucial role in enhancing the capabilities of language models (LMs) for knowledge-intensive tasks. The DSP framework is designed to facilitate sophisticated interactions between LMs and RMs, allowing for the construction of complex, reliable pipelines that systematically break down problems into smaller, manageable transformations. This modular approach enables DSP programs to generate retrieval queries, summarize multiple passages, and provide answers more effectively than simpler retrieve-then-read pipelines or self-ask approaches. By assigning explicit control flow to DSP programs and leveraging the RM for both retrieving relevant passages and constructing demonstrations, DSP achieves significant empirical gains in performance. Additionally, the framework supports the fusion of retrieval results from multiple queries to improve recall and robustness, showcasing the power of combining retrieval and language models in a structured, programmatic manner.

In [53]:
pdf_chunk_sources = output["sources"]

In [54]:
for doc in documents:
    if doc.metadata["page_chunk"] in pdf_chunk_sources:
        print(f"Page chunk {doc.metadata['page_chunk']}")
        print(doc.page_content)

Page chunk 2
DEMONSTRATE –SEARCH –PREDICT : Composing retrieval and language models
QHow many storeys are in...
Q In which city did Akeem 
Ellis play in 2017?
A Ellesmere PortQ When was the discoverer of 
Palomar 4 born?
A 1889Train
Demonstrate
defdemonstrate (x:Example ) -> Example :
x.demos = annotate (x.train, attempt )
return x
defattempt (d:Example ):
d= search(d)
d= predict (d)
if d.pred == d.answer: return d1QHow many storeys are in the castle...
Q When was the discoverer of Palomar 4 born?
A 1889
Hop1 Who discovered Palomar 4?
Psg1 Edwin Hubble discovered Palomar 4...
Hop2 When was Edwin Powell born?
Psg2 Edwin Powell Hubble (1889–1953) was...
Pred 1889
x : ExampleQ In which city did Akeem Ellis play...
A Ellesmere Port
... ...
Pred Waterloo❌Demos“How many storeys are in the 
castle David Gregory inherited?”QHow many storeys are in the...
Demos . . .
Hop1 Which castle did David Gregory inherit?
Psg1 David Gregory inherited Kinnairdy Castle...
Hop2 How many storeys are in Kinnai

In [115]:
# !pip install graphviz

  Using cached graphviz-0.20.1-py3-none-any.whl.metadata (12 kB)
Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)


In [26]:
from pydantic import BaseModel, Field
from typing import List
from openai import OpenAI
import instructor
from graphviz import Digraph
import argparse

class Node(BaseModel):
    id: int
    label: str
    color: str

class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str = "black"

class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)

In [27]:
from IPython.display import display

def visualize_knowledge_graph(kg: KnowledgeGraph):
    dot = Digraph(comment="Knowledge Graph")

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    display(dot)


In [28]:
output

{'answer': 'The Retrieval Model (RM) within the DSP (DEMONSTRATE – SEARCH – PREDICT) framework plays a crucial role in enhancing the capabilities of language models (LMs) for knowledge-intensive tasks. The DSP framework is designed to facilitate sophisticated interactions between LMs and RMs, allowing for the construction of complex, reliable pipelines that systematically break down problems into smaller, manageable transformations. This modular approach enables DSP programs to generate retrieval queries, summarize multiple passages, and provide answers more effectively than simpler retrieve-then-read pipelines or self-ask approaches. By assigning explicit control flow to DSP programs and leveraging the RM for both retrieving relevant passages and constructing demonstrations, DSP achieves significant empirical gains in performance. Additionally, the framework supports the fusion of retrieval results from multiple queries to improve recall and robustness, showcasing the power of combini

- `answer`: Node
- `sources`: Node
    - 0:
    - 1:
    ...
- `query`: Node
- `answer` - `query`: Edge
- `answer` - `0`: Edge
- `answer` - `1`: Edge

In [24]:
structure_schema = """
- `answer`: Node
- `sources`: Node
    - 0:
    - 1:
    ...
- `query`: Node
- `answer` - `query`: Edge
- `answer` - `0`: Edge
- `answer` - `1`: Edge
"""

In [122]:
# !pip install instructor openai

In [29]:
import instructor
from openai import OpenAI

client = instructor.patch(OpenAI())

def generate_graph(answer_output, structure_schema) -> KnowledgeGraph:
    return client.chat.completions.create(
        model="gpt-4-turbo-preview",
        messages=[
            {   "role": "system",
                "content": "You are a parser engine for knowledge graph visualization that transforms dictionary outputs into knowledge graph objects.\
                    The objects in the knowledge graph should be simplifications of the concepts in the question and answer so they can be human readable.\
                    Always use this structure schema for your answers:\n {structure_schema}. Make sure the knowledge graph has the question, the answer and the sources connected to the answer, each its own node.",
            },
            { 
                "role": "user",
                "content": f"Translate this output answer to a knowledge graph object:\n {answer_output}.",
            }
        ],
        response_model=KnowledgeGraph,
    )

In [30]:
answer = output['answer']
sources = output['sources']
answer_output = f"The query: {query}.\n\n The answer: {answer}\n\n The sources: {sources}."

knowledge_graph_object = generate_graph(answer_output, structure_schema)

In [31]:
knowledge_graph_object

KnowledgeGraph(nodes=[Node(id=1, label='Query', color='blue'), Node(id=2, label='Explain simply the Retrieval Model for the DSP framework.', color='green'), Node(id=3, label='Answer', color='blue'), Node(id=4, label='The Retrieval Model (RM) within the DSP (DEMONSTRATE – SEARCH – PREDICT) framework plays a crucial role in enhancing the capabilities of language models (LMs) for knowledge-intensive tasks. The DSP framework is designed to facilitate sophisticated interactions between LMs and RMs, allowing for the construction of complex, reliable pipelines that systematically break down problems into smaller, manageable transformations. This modular approach enables DSP programs to generate retrieval queries, summarize multiple passages, and provide answers more effectively than simpler retrieve-then-read pipelines or self-ask approaches. By assigning explicit control flow to DSP programs and leveraging the RM for both retrieving relevant passages and constructing demonstrations, DSP achi

In [32]:
visualize_knowledge_graph(knowledge_graph_object)